# Static LCA

This notebook explains how to conduct a static (i.e. deterministic; or in other words, ignoring uncertainties) LCA for the pavement structure defined in the case study used in the lectures.

## Content

<span style='background:orange'> 3.1) Single-LCA: My first LCA </span>

<span style='background:orange'> 3.2) Single-LCA: Presentation of the steps needed to calcule the environmental impacts for a product system corresponding to the initial pavement structure. </span>

<span style='background:orange'> 3.3) Multi-LCA </span>


In [1]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import pandas as pd
import numpy as np
from bw2data.query import Filter, Query
from IPython.display import display

In [2]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name
print(ei_name)
print(project_name)

ecoinvent-391-cutoff
2024_STI_BW25_Lecture


In [3]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [4]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

In [5]:
fg_name = "asphalt"

In [6]:
# Assign the foreground database to a variable:
fgdb = bd.Database(fg_name)

## 3.1) Single-LCA: My first LCA with an activity from the ecoinvent database

To perform a LCA, we construct a technosphere matrix, which describes the inputs needed to produce different products (e.g. cars need metal and electricity), and a biosphere matrix, which describes the emissions and resource consumption associated with the production of each product (e.g. car manufacturing releases air emissions). These two matrices come from the life cycle inventory database(s). We also have a functional unit, which is what we are trying to assess, e.g. one car. We then calculate the life cycle inventory (LCI) by first solving the linear system of the technosphere matrix and the functional unit, and then by multiplying the biosphere matrix. To do life cycle impact assessment (LCIA), we multiply the life cycle inventory by a matrix of characterization factors (CFs). A CF is a biophysical quanity that indicates the impact on a certain environmental mechanism per unit of emissions or resource extraction. Adapted from <span style='background:orange'> brightway25/notebooks/Getting Started with Brightway2.ipynb  <span style='background:orange'> </span> or watch the video https://www.youtube.com/watch?v=3GDfNksiY0s  

The full details of the matrix-based LCA is out of the scope of this course. However, if you want learn more about it you can read the  <span style='background:orange'> chapter/book (Heijung, LCA book, etc)... </span>

Brightway has a so-called LCA object. It is instantiated using LCA(args).
The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts.
A second argument that is often passed is an LCIA method, passed using the method tuple.

To illustrate the description above, let's select a simple product system corresponding to an activity that we looked into before, namely the activity 'limestone production, crushed, washed' (kilogram, RoW, None), existing in the ecoinvent database. Its code is 570143a9c955c81d5f8bc1e9bd8d22a1

In [7]:
#Check existing databases
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [8]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

#Assign ei db to a variable
eidb = bd.Database(ei_name)

In [9]:
#Select the activity we are interested in
my_act = eidb.get('570143a9c955c81d5f8bc1e9bd8d22a1')
print(my_act)
my_act.get('reference product')

'limestone production, crushed, washed' (kilogram, RoW, None)


'limestone, crushed, washed'

In [10]:
my_act.as_dict()

{'comment': "Infrastructure data are estimated based on a site visit and sketches of the process. The value of the infrastructure is normalized with a annual production capacity of about 195'000 tons of product per year. The estimated lifespan of the machines is 25 years. Assumed size partition of dust emissions for crushing, screening and transport processes: 50% >10µm (PM10), 45% between 10 µm and 2.5 µm, 5%<2.5 µm. Limestone, crushed, washed is a pre-product for the milling process. 50% of the electricity consumption is covered by the grid and 50% by a hydropower plant nearby. The infrastructure for electricity transportation from power plant to KFN is not included. 50% of the electricity consumption is covered by the grid and 50% by a hydropower plant nearby. The infrastructure for electricity transportation from power plant to KFN is not included.\nGeography:  data are from only one company in Switzerland (KFN), for some exchanges RER-modules have been used as proxy\nTechnology:  

### Functional unit

The first step consists of defining the functional unit (FU). This is done by using a dictionary with keys = activities and values = amounts. In this examples, let's assume we are interested in the production of 1 unit of the reference product. The definition of the FU and respective amount is done as follows:

In [11]:
functional_unit = {my_act: 1}
functional_unit

{'limestone production, crushed, washed' (kilogram, RoW, None): 1}

The next step consists of selecting the LCIA method to be used for calculating the environmental impact scores. In the notebook '2a_Background' we explored the different LCIA methods that come with the databse 'biosphere3'. In this example, we are going to use the method 'CML v4.8 2016', and particularly the impact category 'climate change'.

In [12]:
CML_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
CML_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

The impact category we are interested in is defined as following:

In [13]:
CC_method = ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')

### LCIA scores (i.e. results) calculation

Next, we instantiate our LCA object:

In [14]:
myFirstLCA = bc.LCA(functional_unit, CC_method)

And do the LCI and LCIA calculations:

In [15]:
myFirstLCA.lci()
myFirstLCA.lcia()

Finally, we can print the LCA score:

In [16]:
myFirstLCA.score

0.0031468186429639317

Or in a more complete way:

In [17]:
print("The score is {:f} {} for impact category {}".format(myFirstLCA.score, 
                                                 bd.methods.get(CC_method).get('unit'),
                                                 CC_method[1]))

The score is 0.003147 kg CO2-Eq for impact category climate change


We can also get dataframes for LCA calculation results. 

In [18]:
df = myFirstLCA.to_dataframe()
df

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,689,14277,1.576131e-03,1171,18987,biosphere3,349b29d1-3e58-4c66-98b9-9d1a076efd2e,"Carbon dioxide, fossil",None,kilogram,emission,air,None,ecoinvent-391-cutoff,0c8f3330261952e02d86355874fd2dd8,"diesel, burned in building machine",GLO,megajoule,process,"diesel, burned in building machine"
1,829,1853,1.647161e-04,1649,6563,biosphere3,70ef743b-3ed5-4a6d-b192-fb6d62378555,"Methane, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,5bd4d1d3ea2ad8b6f9288db105968f92,natural gas venting from petroleum/natural gas...,GLO,cubic meter,process,"natural gas, vented"
2,82,3031,1.600458e-04,112,7741,biosphere3,f9749677-9c9f-4678-ab55-c607dfdc2cb9,"Carbon dioxide, fossil",None,kilogram,emission,air::urban air close to ground,None,ecoinvent-391-cutoff,61aa8615096f6a0ba6786e06afe605a1,"heat production, anthracite, at stove 5-15kW",RoW,megajoule,process,"heat, central or small-scale, other than natur..."
3,689,3883,7.391740e-05,1171,8593,biosphere3,349b29d1-3e58-4c66-98b9-9d1a076efd2e,"Carbon dioxide, fossil",None,kilogram,emission,air,None,ecoinvent-391-cutoff,b1b3b47eb0e3d7b0317ae9e38b32c247,"diesel production, petroleum refinery operation",RoW,kilogram,process,diesel
4,686,11949,5.952757e-05,1168,16659,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,c7bceed2927df265f4d635437756b3d9,"sweet gas, burned in gas turbine",GLO,megajoule,process,"sweet gas, burned in gas turbine"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,686,4068,9.297220e-07,1168,8778,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,a5a6266daa144325e454cafbc34bac40,"electricity production, natural gas, conventio...",KR,kilowatt hour,process,"electricity, high voltage"
196,686,12510,9.242229e-07,1168,17220,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,9bcc79119f9fe4087c7c164a7170b093,"electricity production, hard coal",IN-HR,kilowatt hour,process,"electricity, high voltage"
197,82,3767,9.171451e-07,112,8477,biosphere3,f9749677-9c9f-4678-ab55-c607dfdc2cb9,"Carbon dioxide, fossil",None,kilogram,emission,air::urban air close to ground,None,ecoinvent-391-cutoff,4aab528dfde9fd07ce6f1aee4304cf95,"electricity production, oil",MX,kilowatt hour,process,"electricity, high voltage"
198,686,20037,9.157838e-07,1168,24747,biosphere3,aa7cac3a-3625-41d4-bc54-33e2cf11ec46,"Carbon dioxide, fossil",None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,aa4f5d6d66ff569b28464f715e5b251b,"synthetic fuel production, from coal, high tem...",ZA,kilogram,process,"diesel, low-sulfur"


By default, this method looks at the characterized_inventory matrix, and sorts by the top 200 values (using absolute value).

The columns labels are a bit different, as we don't have target and source but instead matrix rows and columns. The meaning of these rows and columns changes from matrix to matrix. The same pattern with 'row_product', 'col_reference_product', and 'row_categories' applies though.

In [19]:
df.columns

Index(['row_index', 'col_index', 'amount', 'row_id', 'col_id', 'row_database',
       'row_code', 'row_name', 'row_location', 'row_unit', 'row_type',
       'row_categories', 'row_product', 'col_database', 'col_code', 'col_name',
       'col_location', 'col_unit', 'col_type', 'col_reference_product'],
      dtype='object')

We can get dataframes for any matrix. In standard LCA, the matrices are:

- inventory
- technosphere_matrix
- biosphere_matrix
- characterization_matrix
- characterized_inventory

Note that for other matrices you will need to specify the row and column mapping dictionaries, see the docstring.

For instance, let's assume we want to get the **'biosphere_matrix'**

In [20]:
myFirstLCA.to_dataframe(matrix_label='biosphere_matrix')

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,333,15850,3.520400e+10,526,20560,biosphere3,379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,"Occupation, mineral extraction site",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,121b570a5c9b8421b33416cb10cfbb27,"mine construction, gold",US,unit,process,"mine infrastructure, gold"
1,332,12243,3.500000e+09,524,16953,biosphere3,69dfa439-8e4e-4cae-bb0c-85a8aa8b9a73,"Occupation, lake, artificial",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,6d8f423f2f91b29a4f1b82cf498a9858,port facilities construction,RoW,unit,process,port facilities
2,332,12091,3.500000e+09,524,16801,biosphere3,69dfa439-8e4e-4cae-bb0c-85a8aa8b9a73,"Occupation, lake, artificial",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,2fd36ed40e1ad80fc2d07b822f6f283d,port facilities construction,RER,unit,process,port facilities
3,333,13685,2.529500e+09,526,18395,biosphere3,379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,"Occupation, mineral extraction site",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,bf79c2917ce1c7c6df3f6e244211b257,"mine construction, gold",CA,unit,process,"mine infrastructure, gold"
4,333,14872,2.360000e+09,526,19582,biosphere3,379ba5c9-5c3a-43d0-8e2d-605ad9c39e46,"Occupation, mineral extraction site",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,f3d7eac4f256e2a49367b2bea0b38b41,"mine infrastructure construction, treatment of...",ZA,unit,process,"mine infrastructure, treatment of sulfidic tai..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,330,7253,7.600000e+06,522,11963,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,ac028074fbd9bee7e47e0f7646e3dd5d,"nuclear power plant construction, boiling wate...",CH,unit,process,"nuclear power plant, boiling water reactor 1000MW"
196,330,7121,7.600000e+06,522,11831,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,92028c4cebf1499c7921bbc531b859df,"nuclear power plant construction, boiling wate...",US,unit,process,"nuclear power plant, boiling water reactor 1000MW"
197,330,18759,7.600000e+06,522,23469,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,bf740244ccab787ac1674ea317bab299,"nuclear power plant construction, boiling wate...",DE,unit,process,"nuclear power plant, boiling water reactor 1000MW"
198,330,18637,7.600000e+06,522,23347,biosphere3,fe9c3a98-a6d2-452d-a9a4-a13e64f1b95b,"Occupation, industrial area",None,square meter-year,natural resource,natural resource::land,None,ecoinvent-391-cutoff,2d2b9c0d438ebf72a85c31cfa574f659,"nuclear power plant construction, boiling wate...",RoW,unit,process,"nuclear power plant, boiling water reactor 1000MW"


In the notebook '2a_background' we saw how the function 'print_recursive_calculation' is used to print the supply chain of an activity. This function can be used to transverse the characterized inventory (i.e., LCA scores):

<span style='background:orange'> Continue based on the website below </span>:
https://github.com/Depart-de-Sentier/teaching-material/blob/main/beginners/Interpretation/Exploration%20functions.ipynb

In [21]:
bwa.print_recursive_calculation(my_act, CC_method)

Fraction of score | Absolute score | Amount | Activity
0001 | 0.003147 |     1 | 'limestone production, crushed, washed' (kilogram, RoW, None)
  0.107 | 0.000336 | 0.003403 | 'market for diesel, burned in building machine' (megajoule, GLO, None)
    0.107 | 0.000336 | 0.003403 | 'diesel, burned in building machine' (megajoule, GLO, None)
      0.0204 | 6.427e-05 | 7.963e-05 | 'market group for diesel' (kilogram, GLO, None)
  0.0572 | 0.0001802 | 0.0001967 | 'market group for electricity, medium voltage' (kilowatt hour, RAS, None)
    0.0341 | 0.0001074 | 0.0001137 | 'market group for electricity, medium voltage' (kilowatt hour, CN, None)
      0.0303 | 9.52e-05 | 9.405e-05 | 'market group for electricity, medium voltage' (kilowatt hour, CN-SGCC, None)


KeyboardInterrupt: 

### Contribution analysis

Quite often, we are interested in knowing the activities that contribute the most to the environmental impacts score. This task is called Contribution analysis and can be done by running the code below:

In [22]:
import bw2analyzer as bwa

In [23]:
ca = bwa.ContributionAnalysis()
results = ca.annotated_top_processes(myFirstLCA, limit=20) #returns a list of tuples: (lca score, supply amount, activity name)

print("Impact category ** {} **".format(CC_method[1].title()))
print(results)

Impact category ** Climate Change **
[(0.0015944183825432857, 0.021590835187152404, 'diesel, burned in building machine' (megajoule, GLO, None)), (0.00017092394554533093, 0.0013990020186153372, 'heat production, anthracite, at stove 5-15kW' (megajoule, RoW, None)), (0.00016485683692315156, 1.0055925519126212e-05, 'natural gas venting from petroleum/natural gas production' (cubic meter, GLO, None)), (7.48150504389357e-05, 0.0003821205838620865, 'diesel production, petroleum refinery operation' (kilogram, RoW, None)), (5.9586519317681226e-05, 0.0008897992546849275, 'sweet gas, burned in gas turbine' (megajoule, GLO, None)), (5.269690383317356e-05, 0.00013828361947590734, 'hard coal mine operation and hard coal preparation' (kilogram, CN, None)), (3.4370656413150856e-05, 4.0479878638776275e-05, 'pig iron production' (kilogram, RoW, None)), (3.24820003283498e-05, 0.0004769750559768772, 'treatment of waste natural gas, sweet, burned in production flare' (megajoule, GLO, None)), (2.969778903

In [24]:
results.as_dict()[0]

AttributeError: 'list' object has no attribute 'as_dict'

we can see the results in a table:

In [25]:
df = pd.DataFrame(results, columns=['CC score', 'Supply amount', 'Activity Name'])
df

,CC score,Supply amount,Activity Name
0,0.001594,0.021591,"[comment, classifications, activity type, acti..."
1,0.000171,0.001399,"[comment, classifications, activity type, acti..."
2,0.000165,0.000010,"[comment, classifications, activity type, acti..."
3,0.000075,0.000382,"[comment, classifications, activity type, acti..."
4,0.000060,0.000890,"[comment, classifications, activity type, acti..."
5,0.000053,0.000138,"[comment, classifications, activity type, acti..."
6,0.000034,0.000040,"[comment, classifications, activity type, acti..."
7,0.000032,0.000477,"[comment, classifications, activity type, acti..."
8,0.000030,0.000258,"[comment, classifications, activity type, acti..."
9,0.000024,0.000018,"[comment, classifications, activity type, acti..."


<span style='background:orange'> See why the results are not the same for the different methods illustrated above </span>

## 3.2) Single-LCA: My second LCA with an activity from the foreground database related to a pavement system

In the previous example we saw how to calculate the environmental impacts of an activity existing in the ecoinvent. In this example, we will see how to calculate the environmental impacts of an foreground activity defined in the foreground database introduced in the notebook '2b_Foreground'. Specifically, we will select the activity 'crushed stone, production and transport'. As depicted in the flowchat bellow, this activity has as inputs the exchanges 'gravel production, crushed', 'market for transport, freight, lorry, unspecified' and 'market for transport, freight, inland waterways, barge' (those are outputs of background activities), and as output the exchange 'crushed stone' (which is the reference product).

<span style='background:orange'> Insert figure with flowchart </span>


### Product system definition

The first step consists of defining the product system. In this example, it comprises one single activity, which is the activity we are interested in. As mentioned above and shown in the excel file, this activity is linked to three background activities defined in the ecoinvent database (you can see that in the column 'database'). Therefore, it is extremely important that the names of the activities and respective reference products are exactly equal to those defined in the ecoinvent databse. Otherwise, you will get an error when running the code.

Let's then select the activity 'crushed stone, production and transport' and assign it to the variable "my_act". We can select it by using the .get() method with the activity code (i.e., crushed_stone) as argument.

In [72]:
my_act = fgdb.get("crushed_stone") #the argument of the method is the code of the activity we are interested in
my_act

'crushed stone, production and transport' (kilogram, NL, None)

In [73]:
bwa.print_recursive_supply_chain(my_act, max_level=1)

1: 'crushed stone, production and transport' (kilogram, NL, None)
  1: 'gravel production, crushed' (kilogram, RoW, None)
  0.025: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
  0.66: 'market for transport, freight, inland waterways, barge' (ton kilometer, RER, None)


### Functional unit

Let's assume we are interested in the production of 1 units of the reference product (i.e., crushed stone)

In [74]:
functional_unit = {my_act: 1}
functional_unit

{'crushed stone, production and transport' (kilogram, NL, None): 1}

### LCIA Methods

The next step consists of selecting the LCIA method to be used for calculating the environmental impact scores. In the notebook '2a_Background' we explored the different LCIA methods that come with the databse 'biosphere3'. In this example, we are going to use the method 'CML v4.8 2016', and particularly the impact category 'Acification'.

In [75]:
CML_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
CML_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

The impact category we are interested in is defined as following:

In [76]:
Acid_method = ('CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)')

### LCIA scores (i.e. results) calculation

Next, we instantiate our LCA object:

In [77]:
mySecondLCA = bc.LCA(functional_unit, Acid_method)

And do the LCI and LCIA calculations:

In [78]:
mySecondLCA.lci()
mySecondLCA.lcia()

Finally, we can print the LCA score:

In [79]:
print("The score is {:f} {} for impact category {}".format(mySecondLCA.score, 
                                                 bd.methods.get(Acid_method).get('unit'),
                                                 Acid_method[1]))

The score is 0.000269 kg SO2-Eq for impact category acidification


In [80]:
df = mySecondLCA.to_dataframe()
df

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,2402,18253,1.551463e-04,4691,22963,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,e43dd9584a807e73dbbcd9d12f26a35e,"transport, freight, inland waterways, barge",RER,ton kilometer,process,"transport, freight, inland waterways, barge"
1,2405,14277,9.804713e-06,4694,18987,biosphere3,c1b91234-6f24-417b-8309-46111d09c457,Nitrogen oxides,None,kilogram,emission,air,None,ecoinvent-391-cutoff,0c8f3330261952e02d86355874fd2dd8,"diesel, burned in building machine",GLO,megajoule,process,"diesel, burned in building machine"
2,946,6871,8.251856e-06,1995,11581,biosphere3,fd7aa71c-508c-480d-81a6-8052aad92646,Sulfur dioxide,None,kilogram,emission,air,None,ecoinvent-391-cutoff,f4e8f9e773206c58d4a9ce05bdfb1e69,"diesel production, petroleum refinery operation",Europe without Switzerland,kilogram,process,diesel
3,944,18253,4.468214e-06,1992,22963,biosphere3,78c3efe4-421c-4d30-82e4-b97ac5124993,Sulfur dioxide,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,e43dd9584a807e73dbbcd9d12f26a35e,"transport, freight, inland waterways, barge",RER,ton kilometer,process,"transport, freight, inland waterways, barge"
4,944,20007,3.612127e-06,1992,24717,biosphere3,78c3efe4-421c-4d30-82e4-b97ac5124993,Sulfur dioxide,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,aebb8b0044d096580f717ac29b5dce3f,"transport, freight, sea, tanker for petroleum",GLO,ton kilometer,process,"transport, freight, sea, tanker for petroleum"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2402,6076,8.140939e-08,4691,10786,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,d24bedfbe8f0bc3405059dd13f83c3fb,"electricity production, hard coal",CN-GS,kilowatt hour,process,"electricity, high voltage"
196,2402,13102,8.095730e-08,4691,17812,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,63bc9f3b9cf61fb5f2b2c2861fde4fd8,"heat and power co-generation, lignite",RU,kilowatt hour,process,"electricity, high voltage"
197,944,10157,7.951846e-08,1992,14867,biosphere3,78c3efe4-421c-4d30-82e4-b97ac5124993,Sulfur dioxide,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,d47c6f40c946e5915657ec3aa90b2c96,"electricity production, hard coal",CN-YN,kilowatt hour,process,"electricity, high voltage"
198,2402,14331,7.948846e-08,4691,19041,biosphere3,77357947-ccc5-438e-9996-95e65e1e1bce,Nitrogen oxides,None,kilogram,emission,air::non-urban air or from high stacks,None,ecoinvent-391-cutoff,798097af2b9c00dc3e391f82d8c7b9d5,"transport, freight, sea, tanker for liquefied ...",GLO,ton kilometer,process,"transport, freight, sea, tanker for liquefied ..."


### Contribution analysis

In [81]:
import bw2analyzer as bwa

In [82]:
ca = bwa.ContributionAnalysis()
results = ca.annotated_top_processes(mySecondLCA, limit=20) #returns a list of tuples: (lca score, supply amount, activity name)

print("Impact category ** {} **".format(Acid_method[1].title()))
print(results)

Impact category ** Acidification **
[(0.00016012896746236, 0.6601971394768142, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (1.035810226525831e-05, 0.01903827659848328, 'diesel, burned in building machine' (megajoule, GLO, None)), (9.00833652549729e-06, 0.006299162417008923, 'diesel production, petroleum refinery operation' (kilogram, Europe without Switzerland, None)), (5.911099075056097e-06, 0.04295536672686118, 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None)), (3.4595308283463968e-06, 0.002955528283951732, 'clinker production' (kilogram, RoW, None)), (2.598923349364906e-06, 0.017867251810533884, 'transport, freight, sea, bulk carrier for dry goods' (ton kilometer, GLO, None)), (2.451782366694148e-06, 0.002802036929741194, 'heat production, at hard coal industrial furnace 1-10MW' (megajoule, RoW, None)), (2.3342125167467973e-06, 4.8575991311652253e-05, 'electricity production, hard coal, at coal mine power plant' (kilowatt hour

In [83]:
df = pd.DataFrame(results, columns=['CC score', 'Supply amount', 'Activity Name'])
df

,CC score,Supply amount,Activity Name
0,0.000160,0.660197,"[comment, classifications, activity type, acti..."
1,0.000010,0.019038,"[comment, classifications, activity type, acti..."
2,0.000009,0.006299,"[comment, classifications, activity type, acti..."
3,0.000006,0.042955,"[comment, classifications, activity type, acti..."
4,0.000003,0.002956,"[comment, classifications, activity type, acti..."
5,0.000003,0.017867,"[comment, classifications, activity type, acti..."
6,0.000002,0.002802,"[comment, classifications, activity type, acti..."
7,0.000002,0.000049,"[comment, classifications, activity type, acti..."
8,0.000002,0.003583,"[comment, classifications, activity type, acti..."
9,0.000002,0.000011,"[comment, classifications, activity type, acti..."


## 3.3) Single-LCA: Revisiting my second LCA with multiple impact categories

In the previous example we saw how to calculate the environmental impact scores for one impact category, namely 'Acification'. In this example we will see how to calculate the environmental impact scores for multiple impact categories

### Product system

The initial step (i.e., Product system definition) is the same as in the previous example. The differences start in the selection of the LCIA methods

In [84]:
my_act = fgdb.get("crushed_stone") #the argument of the method is the code of the activity we are interested in
my_act

'crushed stone, production and transport' (kilogram, NL, None)

### Functional Unit

Let's assume we are interested in the production of 1 units of the reference product (i.e., crushed stone)

In [85]:
functional_unit = {my_act:1}

### LCIA Method

Let's continue to use the method 'CML v4.8 2016'. To select all impact categories, we can use a piece of code that you are already familiar with:

In [86]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

In this step we are going to calculate the environmental impact scores. To do so, we need to do the following:

In [88]:
# Create a lca object
myThirdLca = bc.LCA(functional_unit)

# Builds matrices, solves the system, generates an LCI matrix.
myThirdLca.lci()

results = [] #this list will store the LCA scores
for method in cml_methods:
    myThirdLca.switch_method(method) # the switch method is used to move to the next impact category in the list after the completation of one loop
    myThirdLca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    results.append((method[1].title(), 
                    myThirdLca.score, # Returns the score, i.e. the sum of the characterized inventory
                    bd.methods.get(method).get('unit')))
    print("The score is {:f} {} for the impact category {}".format(myThirdLca.score, 
                                                 bd.methods.get(method).get('unit'),
                                                 method[1].title())
         )

The score is 0.000269 kg SO2-Eq for the impact category Acidification
The score is 0.046649 kg CO2-Eq for the impact category Climate Change
The score is 0.013422 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Freshwater
The score is 27.750110 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Marine
The score is 0.000135 kg 1,4-DCB-Eq for the impact category Ecotoxicity: Terrestrial
The score is 0.552126 megajoule for the impact category Energy Resources: Non-Renewable
The score is 0.000076 kg PO4-Eq for the impact category Eutrophication
The score is 0.025743 kg 1,4-DCB-Eq for the impact category Human Toxicity
The score is 0.000000 kg Sb-Eq for the impact category Material Resources: Metals/Minerals
The score is 0.000000 kg CFC-11-Eq for the impact category Ozone Depletion
The score is 0.000019 kg ethylene-Eq for the impact category Photochemical Oxidant Formation


We can also see the results in a table

In [89]:
#Creates a table with the results
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

,Score,Unit
Impact Category,,
Acidification,2.688610e-04,kg SO2-Eq
Climate Change,4.664932e-02,kg CO2-Eq
Ecotoxicity: Freshwater,1.342170e-02,"kg 1,4-DCB-Eq"
Ecotoxicity: Marine,2.775011e+01,"kg 1,4-DCB-Eq"
Ecotoxicity: Terrestrial,1.346317e-04,"kg 1,4-DCB-Eq"
Energy Resources: Non-Renewable,5.521265e-01,megajoule
Eutrophication,7.598996e-05,kg PO4-Eq
Human Toxicity,2.574321e-02,"kg 1,4-DCB-Eq"
Material Resources: Metals/Minerals,1.020628e-07,kg Sb-Eq


In [ ]:
#Creates a table with the results
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

### Contribution Analysis

In [ ]:
import bw2analyzer as bwa

In [92]:
for method in cml_methods:
    myThirdLca.switch_method(method) #the switch method is used to move to the next impact category in the list after the completation of one loop
    myThirdLca.lcia() # Characterization, i.e. the multiplication of the elements of the LCI matrix with characterization factors from the chosen LCIA method
    
    ca = bwa.ContributionAnalysis()
    results = ca.annotated_top_processes(myThirdLca, limit=5) #returns a list of tuples: (lca score, supply amount, activity name)

    print("Impact category '{}'".format(method[1].title()))
    print(results)

    print("")

Impact category 'Acidification'
[(0.00016012896746236, 0.6601971394768142, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (1.035810226525831e-05, 0.01903827659848328, 'diesel, burned in building machine' (megajoule, GLO, None)), (9.00833652549729e-06, 0.006299162417008923, 'diesel production, petroleum refinery operation' (kilogram, Europe without Switzerland, None)), (5.911099075056097e-06, 0.04295536672686118, 'transport, freight, sea, tanker for petroleum' (ton kilometer, GLO, None)), (3.4595308283463968e-06, 0.002955528283951732, 'clinker production' (kilogram, RoW, None))]

Impact category 'Climate Change'
[(0.02009009606272066, 0.6601971394768142, 'transport, freight, inland waterways, barge' (ton kilometer, RER, None)), (0.002730329779653154, 0.00016654446014661243, 'natural gas venting from petroleum/natural gas production' (cubic meter, GLO, None)), (0.0023905016627848562, 0.002955528283951732, 'clinker production' (kilogram, RoW, None)), (0.0014059

## 3.4) Single-LCA: Revisiting my second LCA with MultiLCA
The MultiLCA allows to calculate the LCA scores for multiple functional units and impact categories.
To do so, we need to create a calculation setup, i.e. a named set of functional units and LCIA methods.

Calculation setups: dictionary with lists of functional units and methods.

In [95]:
list_functional_units = [{my_act:1}] # Let's consider only 1 FU; the same as the one considered in the previous examples

list_methods = cml_methods

In [96]:
bd.calculation_setups['Multile impact categories with MultiLCA'] = {'inv':list_functional_units, 'ia':list_methods}

In [97]:
myMultiLCA = bc.MultiLCA('Multile impact categories with MultiLCA')

In [98]:
def format_results(myMultiLCA):
    formatted_results = []
    for i, scores in enumerate(myMultiLCA.results):  # the results for a fu
        for j, method_key in enumerate(myMultiLCA.methods):  # the result for each method
            demand = list(myMultiLCA.func_units[i].values())[0]
            activity = list(myMultiLCA.func_units[i].keys())[0]
            method = bd.Method(method_key)
            a_result = {
                "activity": repr(activity),
                "demand": demand,
                "method": method_key[0],
                "category": method_key[1],
                "subcategory": method_key[2],
                "score": scores[j],
                "unit": method.metadata["unit"],
            }
            formatted_results.append(a_result)
    return formatted_results

In [103]:
#run the previous function and print the outcome
print(format_results(myMultiLCA)) 

[{'activity': "'crushed stone, production and transport' (kilogram, NL, None)", 'demand': 1, 'method': 'CML v4.8 2016', 'category': 'acidification', 'subcategory': 'acidification (incl. fate, average Europe total, A&B)', 'score': 0.00026886104505952557, 'unit': 'kg SO2-Eq'}, {'activity': "'crushed stone, production and transport' (kilogram, NL, None)", 'demand': 1, 'method': 'CML v4.8 2016', 'category': 'climate change', 'subcategory': 'global warming potential (GWP100)', 'score': 0.046649320142488473, 'unit': 'kg CO2-Eq'}, {'activity': "'crushed stone, production and transport' (kilogram, NL, None)", 'demand': 1, 'method': 'CML v4.8 2016', 'category': 'ecotoxicity: freshwater', 'subcategory': 'freshwater aquatic ecotoxicity (FAETP inf)', 'score': 0.013421704168323079, 'unit': 'kg 1,4-DCB-Eq'}, {'activity': "'crushed stone, production and transport' (kilogram, NL, None)", 'demand': 1, 'method': 'CML v4.8 2016', 'category': 'ecotoxicity: marine', 'subcategory': 'marine aquatic ecotoxici

In [104]:
df_results = pd.DataFrame(format_results(myMultiLCA))
df_results

,activity,demand,method,category,subcategory,score,unit
0,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,acidification,"acidification (incl. fate, average Europe tota...",2.688610e-04,kg SO2-Eq
1,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,climate change,global warming potential (GWP100),4.664932e-02,kg CO2-Eq
2,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,ecotoxicity: freshwater,freshwater aquatic ecotoxicity (FAETP inf),1.342170e-02,"kg 1,4-DCB-Eq"
3,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,ecotoxicity: marine,marine aquatic ecotoxicity (MAETP inf),2.775011e+01,"kg 1,4-DCB-Eq"
4,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,ecotoxicity: terrestrial,terrestrial ecotoxicity (TETP inf),1.346317e-04,"kg 1,4-DCB-Eq"
5,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,energy resources: non-renewable,abiotic depletion potential (ADP): fossil fuels,5.521265e-01,megajoule
6,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,eutrophication,eutrophication (fate not incl.),7.598996e-05,kg PO4-Eq
7,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,human toxicity,human toxicity (HTP inf),2.574321e-02,"kg 1,4-DCB-Eq"
8,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,1.020628e-07,kg Sb-Eq
9,"'crushed stone, production and transport' (kil...",1,CML v4.8 2016,ozone depletion,ozone layer depletion (ODP steady state),6.680045e-10,kg CFC-11-Eq


<span style='background:red'> **Write the code for CA** </span>